In [1]:
%matplotlib inline

# OpenCV-Python utiliza NumPy para el manejo de imágenes
import numpy as np
# cv2 es el módulo python para acceder a OpenCV 
import cv2 as cv
# Usamos las poderosas herramientas de graficación de matplotlib para mostrar imágenes, perfiles, histogramas, etc
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
import math
import sched, time


In [2]:

# Abrimos el archivo
#-------------------
filename_traffic = 'slow_traffic_small.mp4'
filename = 'vtest.avi'

capture = cv.VideoCapture(filename)

if not capture.isOpened:
    print('Falla al abrir el archivo: ' + filename)
    exit(0)

In [3]:
metodo = 'MOG2'
#metodo = 'KNN'
if metodo == 'MOG2':
    backSub = cv.createBackgroundSubtractorMOG2()
else:
    backSub = cv.createBackgroundSubtractorKNN()
    backSub = cv.createBackgroundSubtractorKNN(detectShadows = True)

In [8]:
def calculate_background(frames):    
    new_mask = np.median(frames, axis=0)

    return new_mask.astype(np.uint8)
                

In [9]:
metodo = 'MOG2'
#metodo = 'KNN'
if metodo == 'MOG2':
    backSub = cv.createBackgroundSubtractorMOG2()
else:
    backSub = cv.createBackgroundSubtractorKNN()
    backSub = cv.createBackgroundSubtractorKNN(detectShadows = True)

In [17]:
def substract_backgorund(frames_count, capture, frame_spaces=50):
    ret, frame = capture.read()
    i = 0
    FRAMES = frames_count
    frames_to_add = FRAMES - 1
    frames = np.zeros((FRAMES, frame.shape[0], frame.shape[1], frame.shape[2]))
    should_add_frames = True
    add_frames_step = True
    frames_added = 0
    background = frames[0,:,:,:]
    first_frames = True
    while True:
        # Leemos frames
        ret, frame = capture.read()
        if frame is None:
            break
        
        if first_frames:
            if frames_added < FRAMES:
                frames[frames_added] = frame
                frames_added = frames_added + 1
                background = calculate_background(frames)
            else:
                first_frames = False
                frames_added = 0

        if i % frame_spaces == 0 and not first_frames:
            if frames_added < frames_to_add:
                frames[frames_added] = frame
                frames_added = frames_added + 1
                background = calculate_background(frames)
            else:
                frames[frames_added] = background
                frames_added = 0
            
        i = i + 1
  
        new_frame = cv.subtract(background, frame)
        fgMask = backSub.apply(frame)
        cv.imshow('Frame', frame)
        cv.imshow('Naive method', new_frame)
        cv.imshow('background', background)
        cv.imshow('FG Mask', fgMask)

        # Corremos hasta que termine o apriete escape
        keyboard = cv.waitKey(30)
        if keyboard == 'q' or keyboard == 27:
            break
        
    cv.destroyAllWindows()
    capture.release()

In [18]:
substract_backgorund(6, capture, 50)

KeyboardInterrupt: 

Vemos que el metodo naive se comporta muy bien respecto a otros metodos de substracción de fondo, incluso mejorando respecto al ruido. El calculo del fondo se realiza cada 50 frames, y se agrega un frame cada 50 al buffer de frames a considerar para calcular la media. De este modo se favorece al fondo respecto a los objetos que te mueven. Los campos cantidad de frames y espacio entre frames a actualizar para el fondo son configurables por parametros